In [1]:
import yfinance as yf
import pandas as pd
from df_preprocessing import convert_to_pct
import os

import time

In [ ]:
list_stocks = ['AAPL', 'LMT', 'BA', 'T', 'NFLX', 'TWD', 'ASML', 'INTC', 'NVDA', 'AMD', 'MSFT', 'AMZN', 'TSLA', 'MLTX']
new_list = ['AAPL', 'INTC']

In [24]:
acl_18_ext = ['AAPL', 'ABB', 'ABBV', 'AEP', 'AMGN', 'AMZN', 'BA', 'BAC', 'BBL', 'BCH', 'BHP', 'BP', 'BRK-A', 'BSAC', 'BUD', 'C', 'CAT', 'CELG', 'CHL', 'CHTR', 'CMCSA', 'CODI', 'CSCO', 'CVX', 'D', 'DHR', 'DIS', 'DUK', 'EXC', 'FB', 'GD', 'GE', 'GOOG', 'HD', 'HON', 'HRG', 'HSBC', 'IEP', 'INTC', 'JNJ', 'JPM', 'KO', 'LMT', 'MA', 'MCD', 'MDT', 'MMM', 'MO', 'MRK', 'MSFT', 'NEE', 'NGG', 'NVS', 'ORCL', 'PCG', 'PCLN', 'PEP', 'PFE', 'PG', 'PICO', 'PM', 'PPL', 'PTR', 'RDS-B', 'REX', 'SLB', 'SNP', 'SNY', 'SO', 'SPLP', 'SRE', 'T', 'TM', 'TOT', 'TSM', 'UL', 'UN', 'UNH', 'UPS', 'UTX', 'V', 'VZ', 'WFC', 'WMT', 'XOM']

# MARKET INDEX

In [ ]:
market_index = '^GSPC'

In [ ]:
mk_ticker = yf.Ticker(market_index)

In [ ]:
#mk_ticker.info
sp_500_data = mk_ticker.history(period='max', interval='1d')

In [ ]:
#processing
sp_500_data = sp_500_data.reset_index()
sp_500_data = sp_500_data.rename(columns= lambda x : 'index_' + x.lower() )
sp_500_data.drop(['index_dividends', 'index_stock splits', 'index_open'], axis=1, inplace=True)

#trim down to 1d inerval time
index_dt = pd.to_datetime(sp_500_data['index_date'])
sp_500_data['index_date'] = index_dt.dt.strftime('%Y-%m-%d')

#convert to pct change
index_pct_list = ['index_high', 'index_low', 'index_close']
convert_to_pct(sp_500_data, index_pct_list, drop_inplace=True, drop_NA=True)

In [ ]:
display(sp_500_data.describe())
display(sp_500_data[-30:])

# stock list data

In [3]:
stock_ticker_dict = {}

for stock_name in acl_18_ext:
    
    c_ticker = yf.Ticker(stock_name)
    c_data = c_ticker.history(start='2016-01-01', end='2024-04-01', interval='1d')
    stock_ticker_dict[stock_name] = [c_ticker, c_data]


    time.sleep(0.5)

ABB: No timezone found, symbol may be delisted
BBL: No timezone found, symbol may be delisted
CELG: No timezone found, symbol may be delisted
CHL: No timezone found, symbol may be delisted
FB: No timezone found, symbol may be delisted
HRG: No price data found, symbol may be delisted (1d 2016-01-01 -> 2024-04-01)
PCLN: No price data found, symbol may be delisted (1d 2016-01-01 -> 2024-04-01)
PICO: No timezone found, symbol may be delisted
PTR: No timezone found, symbol may be delisted
RDS-B: No timezone found, symbol may be delisted
SNP: No timezone found, symbol may be delisted
TOT: No timezone found, symbol may be delisted
UN: No timezone found, symbol may be delisted
UTX: No timezone found, symbol may be delisted


In [25]:
# stock_ticker_dict['ABB'][1].empty

temp_l = []

for stock_name in acl_18_ext:
    if not stock_ticker_dict[stock_name][1].empty:
        temp_l.append(stock_name)

acl_18_ext = temp_l


In [27]:
for stock_name in acl_18_ext:
    
    #original
    stock_ticker_dict[stock_name][1].drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].reset_index()
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].rename(columns=lambda x: x.lower())

    #datetime info change
    temp_dt = pd.to_datetime(stock_ticker_dict[stock_name][1].date)
    stock_ticker_dict[stock_name][1]['date'] = temp_dt.dt.strftime('%Y-%m-%d')

    #pct
    temp_data = stock_ticker_dict[stock_name][1].copy()
    convert_to_pct(temp_data, ['open', 'high', 'low', 'close'], drop_NA=True)
    temp_data = temp_data.reset_index().drop('index', axis=1)
    stock_ticker_dict[stock_name].append(temp_data)

    

    # #pct + index
    # index_match_df = sp_500_data[sp_500_data['index_date'].isin(temp_data['date'])].reset_index().drop('index', axis=1)
    # merged_df = pd.concat([temp_data, index_match_df], axis=1, join='outer')
    # merged_df.drop(['index_date'], axis=1, inplace=True)
    # stock_ticker_dict[stock_name].append(merged_df)
    

In [ ]:
# #test
# dtx = sp_500_data[sp_500_data['index_date'].isin(stock_ticker_dict['AAPL'][2]['date'])].reset_index().drop('index', axis=1)
# dtx_m = pd.concat([stock_ticker_dict['AAPL'][2], dtx], axis=1, join='outer')
# # dtx = dtx.rename(columns={'index_date':'date'})
# # dtx.date = pd.to_datetime(dtx.date)
# # stock_ticker_dict['AAPL'][2].date = pd.to_datetime(stock_ticker_dict['AAPL'][2].date)
# # dtx_m = pd.merge(stock_ticker_dict['AAPL'][2], index_match_df, on='date', how='left')

In [30]:
# display(dtx[-10:])
# display(dtx_m)
acl_18_ext[0]

'AAPL'

In [29]:

display(stock_ticker_dict[acl_18_ext[0]][1])
display(stock_ticker_dict[acl_18_ext[0]][2])
# display(stock_ticker_dict[list_stocks[0]][3])

,date,open,high,low,close,volume
0,2016-01-04,23.324093,23.951464,23.185435,23.946917,270597600
1,2016-01-05,24.037844,24.060574,23.278635,23.346827,223164000
2,2016-01-06,22.858110,23.269539,22.701268,22.889933,273829600
3,2016-01-07,22.430770,22.760366,21.919327,21.923872,324377600
4,2016-01-08,22.401225,22.528517,21.994344,22.039804,283192000
...,...,...,...,...,...,...
2068,2024-03-22,171.759995,173.050003,170.059998,172.279999,71106600
2069,2024-03-25,170.570007,171.940002,169.449997,170.850006,54288300
2070,2024-03-26,170.000000,171.419998,169.580002,169.710007,57388400
2071,2024-03-27,170.410004,173.600006,170.110001,173.309998,60273300


,date,open,high,low,close,volume,open_pct_change,high_pct_change,low_pct_change,close_pct_change
0,2016-01-05,24.037844,24.060574,23.278635,23.346827,223164000,3.060145,0.455548,0.401979,-2.505918
1,2016-01-06,22.858110,23.269539,22.701268,22.889933,273829600,-4.907820,-3.287684,-2.480244,-1.956985
2,2016-01-07,22.430770,22.760366,21.919327,21.923872,324377600,-1.869532,-2.188150,-3.444484,-4.220461
3,2016-01-08,22.401225,22.528517,21.994344,22.039804,283192000,-0.131714,-1.018651,0.342241,0.528796
4,2016-01-11,22.496691,22.517148,22.126178,22.396675,198957600,0.426163,-0.050466,0.599401,1.619210
...,...,...,...,...,...,...,...,...,...,...
2067,2024-03-22,171.759995,173.050003,170.059998,172.279999,71106600,-2.987861,-2.501551,-0.456567,0.531017
2068,2024-03-25,170.570007,171.940002,169.449997,170.850006,54288300,-0.692820,-0.641433,-0.358697,-0.830040
2069,2024-03-26,170.000000,171.419998,169.580002,169.710007,57388400,-0.334178,-0.302434,0.076722,-0.667252
2070,2024-03-27,170.410004,173.600006,170.110001,173.309998,60273300,0.241179,1.271735,0.312536,2.121260


In [ ]:
save_raw = False
save_pct = True
save_pct_index = True


#export csv
folder_raw = 'data_us_raw/'
folder_pct = 'data_us_pct/'
folder_pct_index = 'stock_us_pctindex/'

if not os.path.exists(folder_raw):
    os.makedirs(folder_raw)

if not os.path.exists(folder_pct):
    os.makedirs(folder_pct)

if not os.path.exists(folder_pct_index):
    os.makedirs(folder_pct_index)

for stock_name in new_list:

    if save_raw:
        file_ori = folder_raw + stock_name+'_raw.csv'
        stock_ticker_dict[stock_name][1].to_csv(file_ori, index=False)

    if save_pct:
        file_pct = folder_pct + stock_name+'_pct.csv'
        stock_ticker_dict[stock_name][2].to_csv(file_pct, index=False)

    if save_pct_index:
        file_pct_index = folder_pct_index + stock_name + '_pct_index.csv'
        stock_ticker_dict[stock_name][3].to_csv(file_pct_index, index=False)

1) apple 
2) Lockheed Martin 
3) Boeing 
4) AT&T
5) Netflix
6) tsmc
7) asml
8) intel
9) nvidia 
10) amd
11. HLX
12. XPOF
13. WGO
14. MLTX
15. WW
16. INSW
17. MLTX
18. msft
19. intc
20.  jpmc
21. amzn
22. tsla